In [1]:
import numpy as np
import pandas as pd
from boruta import BorutaPy
from sklearn.datasets import load_diabetes
from sklearn.metrics import accuracy_score 
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [2]:
%cd Z:\PPMI_Data\Excel_Data\NonMotors\Feature_CpG_Gene_Association\SymbolDigit_Processed_Files
data = pd.read_csv('SymbDigiCateg_Methylome_APPRDXFilter.csv')
data.head(2)

Z:\PPMI_Data\Excel_Data\NonMotors\Feature_CpG_Gene_Association\SymbolDigit_Processed_Files


,Sentrix,PATNO,HYS,Gender,APPRDX,Symbol_Digit,cg20815302,cg25795414,cg00959375,cg07781847,...,cg03814399,cg14792798,cg16910097,cg08750857,cg17933893,cg06917935,cg02794319,cg10702286,cg10353174,cg19590050
0,200991620021_R01C01,3001,2,1,1,1,0.907957,0.868910,0.752844,0.827413,...,0.880496,0.928546,0.900567,0.877800,0.836823,0.216268,0.234350,0.681550,0.103758,0.837890
1,200973410167_R04C01,3007,2,1,1,2,0.918626,0.865257,0.798170,0.823846,...,0.877306,0.912956,0.935591,0.942104,0.750613,0.254265,0.290337,0.760554,0.075480,0.833824


In [3]:
X = data.drop(['PATNO','Sentrix','HYS','Gender','Symbol_Digit'], axis = 1)
y = data['APPRDX']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .2, random_state = 1) 

In [4]:
rf_all_features = RandomForestClassifier(random_state=1)
rf_all_features.fit(X_train, y_train)

RandomForestClassifier(random_state=1)

In [5]:
accuracy_score(y_test, rf_all_features.predict(X_test))

0.7282608695652174

In [6]:
rfc = RandomForestClassifier(random_state=1)
boruta_selector = BorutaPy(rfc, n_estimators= 'auto', verbose=1, random_state=1)
boruta_selector.fit(np.array(X_train), np.array(y_train))  

Iteration: 1 / 100
Iteration: 2 / 100
Iteration: 3 / 100
Iteration: 4 / 100
Iteration: 5 / 100
Iteration: 6 / 100
Iteration: 7 / 100
Iteration: 8 / 100
Iteration: 9 / 100


BorutaPy finished running.

Iteration: 	10 / 100
Confirmed: 	1
Tentative: 	0
Rejected: 	192582


BorutaPy(estimator=RandomForestClassifier(n_estimators=14,
                                          random_state=RandomState(MT19937) at 0x2A3B94C0C40),
         n_estimators='auto',
         random_state=RandomState(MT19937) at 0x2A3B94C0C40, verbose=1)

In [7]:
# print("Ranking: ",boruta_selector.ranking_)          
print("No. of significant features: ", boruta_selector.n_features_)

No. of significant features:  1


In [8]:
selected_rf_features = pd.DataFrame({'CpG':list(X_train.columns),
                                       'Ranking':boruta_selector.ranking_})
selected_rf_features = selected_rf_features.sort_values(by='Ranking')

In [9]:
Top50 = selected_rf_features.iloc[:51, :]
Top50.head(2)

,CpG,Ranking
0,APPRDX,1
117260,cg19816560,2


In [10]:
Top50.to_csv ('SymbDigi_Boruta_Top50.csv', index = False)
selected_rf_features.to_csv ('SymbDigi_Boruta_All.csv', index = False)

#Identify a string in dataframe
substring = 'cg15328505'
selected_rf_features[selected_rf_features.apply(lambda row: row.astype(str).str.contains(substring, case=False).any(), axis=1)]
